# Helper Functions Showcase

This notebook demonstrates all pollywog helper functions for common calculation patterns. You'll learn:
- Mathematical helpers (Sum, Product, Average)
- Weighting helpers (WeightedAverage, WeightedSum)
- Transformation helpers (Scale, Normalize)
- Classification helper (CategoryFromThresholds)
- Dual-mode usage (with/without `name` parameter)

In [ ]:
import pollywog as pw
from pollywog.core import CalcSet, Number, Category
from pollywog.helpers import (
    Sum, Product, Average,
    WeightedAverage, WeightedSum,
    Scale, Normalize,
    CategoryFromThresholds
)
from pollywog.display import display_calcset, set_theme

set_theme("light")

## Understanding Dual-Mode Helpers

All helpers have two modes:

**Mode 1: With `name` parameter** → Returns complete Item object
```python
Sum(["Au", "Ag", "Cu"], name="total")  # Returns Number object
```

**Mode 2: Without `name`** → Returns expression string (for composition)
```python
Sum(["Au", "Ag", "Cu"])  # Returns "([Au] + [Ag] + [Cu])"
```

## 1. Sum: Add Multiple Variables

Calculate sum of multiple variables.

In [ ]:
# Mode 1: Returns Number object
sum_calc = Sum(["Au", "Ag", "Cu"], name="total_metals")

# Mode 2: Returns expression string for composition
sum_expr = Sum(["Au", "Ag", "Cu"])
custom_calc = Number("metals_times_two", f"2 * {sum_expr}")

calcset_sum = CalcSet([sum_calc, custom_calc])
display_calcset(calcset_sum)

print(f"\nMode 2 expression: {sum_expr}")
print("This allows building complex formulas by combining helpers!")

## 2. Product: Multiply Multiple Variables

Calculate product of multiple variables.

In [ ]:
# Tonnage calculation: volume × density
tonnage = Product(["block_volume", "density"], name="tonnage")

# Contained metal: grade × tonnage × recovery
contained_metal = Product(
    ["Au", "tonnage", "recovery"],
    name="contained_Au",
    comment="Grade × tonnage × recovery"
)

# Net smelter return: grade × recovery × price × adjustment
nsr = Product(
    ["Au", "recovery", "price", "adjustment_factor"],
    name="NSR",
    comment="Net smelter return per tonne"
)

calcset_product = CalcSet([tonnage, contained_metal, nsr])
display_calcset(calcset_product)

## 3. Average: Arithmetic Mean

Calculate simple average of multiple variables.

In [ ]:
# Average of multiple assay methods
avg_au = Average(
    ["Au_fire_assay", "Au_screen", "Au_leach"],
    name="Au_average",
    comment="Average of three assay methods"
)

# Average grade across adjacent blocks
smoothed = Average(
    ["Au_center", "Au_north", "Au_south", "Au_east", "Au_west"],
    name="Au_smoothed",
    comment="Smoothed using neighboring blocks"
)

calcset_average = CalcSet([avg_au, smoothed])
display_calcset(calcset_average)

## 4. WeightedAverage: Domain Proportion Weighting

**Critical for multi-domain modeling!** Calculate weighted average based on domain proportions.

In [ ]:
# Domain-weighted grade (most common use case!)
au_composite = WeightedAverage(
    variables=["Au_oxide", "Au_sulfide", "Au_transition"],
    weights=["prop_oxide", "prop_sulfide", "prop_transition"],
    name="Au_composite",
    comment="Domain-weighted gold grade"
)

# Recovery weighted by material proportions
recovery_composite = WeightedAverage(
    variables=["recovery_HG", "recovery_LG", "recovery_waste"],
    weights=["prop_HG", "prop_LG", "prop_waste"],
    name="recovery_final",
    comment="Weighted by domain proportions"
)

# Density weighted by lithology proportions
density_composite = WeightedAverage(
    variables=["density_granite", "density_basalt", "density_sediment"],
    weights=["prop_granite", "prop_basalt", "prop_sediment"],
    name="density_final",
    comment="Lithology-weighted bulk density"
)

calcset_weighted = CalcSet([au_composite, recovery_composite, density_composite])
display_calcset(calcset_weighted)

print("\n💡 WeightedAverage is THE most important helper for multi-domain resource models!")

## 5. WeightedSum: Proportional Contribution

Similar to WeightedAverage but doesn't normalize (doesn't divide by sum of weights).

In [ ]:
# Tonnage contribution from each domain
tonnage_contribution = WeightedSum(
    variables=["tonnage_oxide", "tonnage_sulfide"],
    weights=["prop_oxide", "prop_sulfide"],
    name="total_tonnage",
    comment="Proportional tonnage from each domain"
)

calcset_weighted_sum = CalcSet([tonnage_contribution])
display_calcset(calcset_weighted_sum)

print("\n📊 Use WeightedSum when you want proportional contributions without normalization.")
print("📊 Use WeightedAverage when you want a true average (most common).")

## 6. Scale: Multiply by Constant Factor

Apply a constant multiplier to a variable.

In [ ]:
# Apply dilution factor
au_diluted = Scale("Au_est", 0.95, name="Au_diluted", comment="5% dilution")

# Apply recovery factor
au_recovered = Scale("Au_diluted", 0.88, name="Au_recovered", comment="88% recovery")

# Convert g/t to oz/t
au_ozt = Scale("Au", 0.029166667, name="Au_ozt", comment="Convert g/t to oz/t")

# Unit conversion: tonnes to pounds
cu_lbs = Scale("Cu_tonnes", 2204.62, name="Cu_lbs", comment="Tonnes to pounds")

calcset_scale = CalcSet([au_diluted, au_recovered, au_ozt, cu_lbs])
display_calcset(calcset_scale)

## 7. Normalize: Scale to Sum to 1

Normalize a set of values so they sum to 1 (useful for proportions).

In [ ]:
# Normalize domain proportions (ensure they sum to 1)
normalized_props = Normalize(
    ["prop_oxide", "prop_sulfide", "prop_transition"],
    name="normalized",
    comment="Proportions normalized to sum to 1"
)

# Returns three variables: normalized_oxide, normalized_sulfide, normalized_transition

calcset_normalize = CalcSet(normalized_props)
display_calcset(calcset_normalize)

print("\n⚠️ Normalize returns a LIST of Number objects (one per input variable)")
print("Each output has suffix matching the input variable name.")

## 8. CategoryFromThresholds: Grade Classification

Create categorical classifications from numeric thresholds.

In [ ]:
# Simple grade classification
ore_class = CategoryFromThresholds(
    variable="Au",
    thresholds=[0.5, 2.0],
    categories=["waste", "low_grade", "high_grade"],
    name="ore_class",
    comment="Classify by Au cutoffs"
)

# Multi-level classification
detailed_class = CategoryFromThresholds(
    variable="Cu",
    thresholds=[0.1, 0.3, 0.6, 1.0],
    categories=["waste", "marginal", "low_grade", "medium_grade", "high_grade"],
    name="Cu_class",
    comment="5-class copper classification"
)

# Recovery classification
recovery_class = CategoryFromThresholds(
    variable="recovery",
    thresholds=[0.7, 0.85],
    categories=["poor", "good", "excellent"],
    name="recovery_class",
    comment="Metallurgical recovery classification"
)

calcset_category = CalcSet([ore_class, detailed_class, recovery_class])
display_calcset(calcset_category)

print("\n📝 CategoryFromThresholds logic:")
print("   value < threshold[0] → category[0]")
print("   threshold[0] ≤ value < threshold[1] → category[1]")
print("   threshold[1] ≤ value < threshold[2] → category[2]")
print("   etc.")

## 9. Composing Helpers: Building Complex Expressions

Use helpers without `name` to build complex formulas.

In [ ]:
# Build complex expression by composing helpers

# Average of domain grades, then scale by recovery
au_expr = Average(["Au_oxide", "Au_sulfide"])
recoverable = Number("Au_recoverable", f"{au_expr} * [recovery]")

# NSR calculation: (Au_value + Ag_value) × recovery - costs
au_value = Scale("Au", 50)  # Returns expression, not Number
ag_value = Scale("Ag", 0.8)
gross_value = Sum([au_value, ag_value])
nsr = Number("NSR", f"({gross_value}) * [recovery] - [cost]")

# Equivalent copper: Au_eq + Cu where Au is scaled by price ratio
au_eq = Scale("Au", 31.0)  # 31 g/t Au = 1% Cu equivalent
cu_equivalent = Number("Cu_equiv", f"{au_eq} + [Cu]")

calcset_composed = CalcSet([recoverable, nsr, cu_equivalent])
display_calcset(calcset_composed)

print("\n🎯 Composition is powerful!")
print("Build complex calculations by combining simple helpers.")

## 10. Complete Real-World Example

A realistic resource estimation workflow using multiple helpers.

In [ ]:
# Complete post-processing workflow
resource_workflow = CalcSet([
    # 1. Combine domain estimates (weighted average)
    WeightedAverage(
        variables=["Au_oxide", "Au_sulfide", "Au_transition"],
        weights=["prop_oxide", "prop_sulfide", "prop_transition"],
        name="Au_composite",
        comment="Domain-weighted estimate"
    ),
    
    # 2. Apply mining factors (scale)
    Scale("Au_composite", 0.95, name="Au_diluted", comment="5% dilution"),
    Scale("Au_diluted", 0.88, name="Au_recovered", comment="88% recovery"),
    
    # 3. Calculate tonnage (product)
    Product(["block_volume", "density"], name="tonnage", comment="Block tonnage"),
    
    # 4. Metal content
    Product(
        ["tonnage", "Au_recovered"],
        name="Au_grams",
        comment="Contained gold (grams)"
    ),
    Scale("Au_grams", 0.03215075, name="Au_ounces", comment="Convert to troy oz"),
    
    # 5. Economic value
    Product(
        ["Au_ounces", "gold_price"],
        name="gross_value",
        comment="Gross value (USD)"
    ),
    
    # 6. Classification (thresholds)
    CategoryFromThresholds(
        variable="Au_recovered",
        thresholds=[0.3, 1.0, 3.0],
        categories=["waste", "low_grade", "medium_grade", "high_grade"],
        name="resource_class",
        comment="Resource classification"
    ),
])

display_calcset(resource_workflow)

print(f"\nComplete workflow created with {len(resource_workflow.items)} calculations!")
print("This would have taken 15-20 minutes to build manually in Leapfrog.")
print("With helpers: a few lines of Python code. ✨")

resource_workflow.to_lfcalc("resource_workflow.lfcalc")

## Summary: Helper Functions Quick Reference

| Helper | Purpose | Example |
|--------|---------|--------|
| **Sum** | Add variables | `Sum(["Au", "Ag"], name="total")` |
| **Product** | Multiply variables | `Product(["grade", "recovery"], name="payable")` |
| **Average** | Arithmetic mean | `Average(["Au_1", "Au_2"], name="Au_avg")` |
| **WeightedAverage** | Domain-weighted average | `WeightedAverage(variables=[...], weights=[...])` |
| **WeightedSum** | Proportional sum | `WeightedSum(variables=[...], weights=[...])` |
| **Scale** | Multiply by constant | `Scale("Au", 0.95, name="Au_diluted")` |
| **Normalize** | Scale to sum to 1 | `Normalize(["prop1", "prop2"], name="norm")` |
| **CategoryFromThresholds** | Classify by thresholds | `CategoryFromThresholds("Au", [0.5, 2.0], [...])` |

**Key patterns:**
- ✅ **With `name`**: Get complete Item object
- ✅ **Without `name`**: Get expression string for composition
- ✅ All helpers auto-wrap variables in brackets: `"Au"` → `"[Au]"`
- ✅ Normalize returns LIST of items (one per input)
- ⭐ WeightedAverage is THE critical helper for multi-domain models